In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import os
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.metrics.cluster import homogeneity_score
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

In [2]:
df_metrics = pd.DataFrame(columns=['ARI_Louvain','ARI_kmeans','ARI_HC',
                                   'AMI_Louvain','AMI_kmeans','AMI_HC',
                                   'Homogeneity_Louvain','Homogeneity_kmeans','Homogeneity_HC'])

In [3]:
workdir = './output/'
path_fm = os.path.join(workdir,'feature_matrices/')
path_clusters = os.path.join(workdir,'clusters/')
path_metrics = os.path.join(workdir,'metrics/')
os.system('mkdir -p '+path_clusters)
os.system('mkdir -p '+path_metrics)

0

In [4]:
metadata = pd.read_csv('./input/metadata.tsv',sep='\t',index_col=0)
num_clusters = len(np.unique(metadata['label']))
print(num_clusters)

10


In [5]:
files = [x for x in os.listdir(path_fm) if x.startswith('FM')]
len(files)

13

In [6]:
files

['FM_ChromVAR_buenrostro2018bulkpeaks_kmers.rds',
 'FM_ChromVAR_buenrostro2018bulkpeaks_motifs.rds',
 'FM_cisTopic_buenrostro2018bulkpeaks.rds',
 'FM_Cusanovich2018_buenrostro2018bulkpeaks.rds',
 'FM_Control_buenrostro2018bulkpeaks.rds',
 'FM_GeneScoring_buenrostro2018bulkpeaks.rds',
 'FM_Scasat_buenrostro2018bulkpeaks.rds',
 'FM_scABC_buenrostro2018bulkpeaks.rds',
 'FM_Cicero_buenrostro2018bulkpeaks.rds',
 'FM_ChromVAR_buenrostro2018bulkpeaks_kmers_pca.rds',
 'FM_ChromVAR_buenrostro2018bulkpeaks_motifs_pca.rds',
 'FM_GeneScoring_buenrostro2018bulkpeaks_pca.rds',
 'FM_Cicero_buenrostro2018bulkpeaks_pca.rds']

In [7]:
def getNClusters(adata,n_cluster,range_min=0,range_max=3,max_steps=20):
    this_step = 0
    this_min = float(range_min)
    this_max = float(range_max)
    while this_step < max_steps:
        print('step ' + str(this_step))
        this_resolution = this_min + ((this_max-this_min)/2)
        sc.tl.louvain(adata,resolution=this_resolution)
        this_clusters = adata.obs['louvain'].nunique()
        
        print('got ' + str(this_clusters) + ' at resolution ' + str(this_resolution))
        
        if this_clusters > n_cluster:
            this_max = this_resolution
        elif this_clusters < n_cluster:
            this_min = this_resolution
        else:
            return(this_resolution, adata)
        this_step += 1
    
    print('Cannot find the number of clusters')
    print('Clustering solution from last iteration is used:' + str(this_clusters) + ' at resolution ' + str(this_resolution))

In [8]:
for file in files:
    file_split = file.split('_')
    method = file_split[1]
    dataset = file_split[2].split('.')[0]
    if(len(file_split)>3):
        method = method + '_' + '_'.join(file_split[3:]).split('.')[0]
    print(method)

    pandas2ri.activate()
    readRDS = robjects.r['readRDS']
    df_rds = readRDS(os.path.join(path_fm,file))
    fm_mat = pandas2ri.ri2py(robjects.r['data.frame'](robjects.r['as.matrix'](df_rds)))
    fm_mat.fillna(0,inplace=True)
    fm_mat.columns = metadata.index
    
    adata = sc.AnnData(fm_mat.T)
    adata.var_names_make_unique()
    adata.obs = metadata.loc[adata.obs.index,]
    df_metrics.loc[method,] = ""
    #Louvain
    sc.pp.neighbors(adata, n_neighbors=15,use_rep='X')
#     sc.tl.louvain(adata)
    getNClusters(adata,n_cluster=num_clusters)
    #kmeans
    kmeans = KMeans(n_clusters=num_clusters, random_state=2019).fit(adata.X)
    adata.obs['kmeans'] = pd.Series(kmeans.labels_,index=adata.obs.index).astype('category')
    #hierachical clustering
    hc = AgglomerativeClustering(n_clusters=num_clusters).fit(adata.X)
    adata.obs['hc'] = pd.Series(hc.labels_,index=adata.obs.index).astype('category')
    #clustering metrics
    
    #adjusted rank index
    ari_louvain = adjusted_rand_score(adata.obs['label'], adata.obs['louvain'])
    ari_kmeans = adjusted_rand_score(adata.obs['label'], adata.obs['kmeans'])
    ari_hc = adjusted_rand_score(adata.obs['label'], adata.obs['hc'])
    #adjusted mutual information
    ami_louvain = adjusted_mutual_info_score(adata.obs['label'], adata.obs['louvain'],average_method='arithmetic')
    ami_kmeans = adjusted_mutual_info_score(adata.obs['label'], adata.obs['kmeans'],average_method='arithmetic')   
    ami_hc = adjusted_mutual_info_score(adata.obs['label'], adata.obs['hc'],average_method='arithmetic')
    #homogeneity
    homo_louvain = homogeneity_score(adata.obs['label'], adata.obs['louvain'])
    homo_kmeans = homogeneity_score(adata.obs['label'], adata.obs['kmeans'])
    homo_hc = homogeneity_score(adata.obs['label'], adata.obs['hc'])

    df_metrics.loc[method,['ARI_Louvain','ARI_kmeans','ARI_HC']] = [ari_louvain,ari_kmeans,ari_hc]
    df_metrics.loc[method,['AMI_Louvain','AMI_kmeans','AMI_HC']] = [ami_louvain,ami_kmeans,ami_hc]
    df_metrics.loc[method,['Homogeneity_Louvain','Homogeneity_kmeans','Homogeneity_HC']] = [homo_louvain,homo_kmeans,homo_hc] 
    adata.obs[['louvain','kmeans','hc']].to_csv(os.path.join(path_clusters ,method + '_clusters.tsv'),sep='\t')

ChromVAR_kmers


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 9 at resolution 1.5
step 1
got 14 at resolution 2.25
step 2
got 10 at resolution 1.875
ChromVAR_motifs


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 10 at resolution 1.5
cisTopic


...

step 0
got 12 at resolution 1.5
step 1
got 7 at resolution 0.75
step 2
got 8 at resolution 1.125
step 3
got 9 at resolution 1.3125
step 4
got 10 at resolution 1.40625
GeneScoring_pca


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 14 at resolution 1.5
step 1
got 8 at resolution 0.75
step 2
got 12 at resolution 1.125
step 3
got 9 at resolution 0.9375
step 4
got 10 at resolution 1.03125
Cicero_pca


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 9 at resolution 1.5
step 1
got 19 at resolution 2.25
step 2
got 11 at resolution 1.875
step 3
got 9 at resolution 1.6875
step 4
got 12 at resolution 1.78125
step 5
got 12 at resolution 1.734375
step 6
got 11 at resolution 1.7109375
step 7
got 10 at resolution 1.69921875


In [9]:
df_metrics.to_csv(path_metrics+'clustering_scores.csv')

In [10]:
df_metrics

,ARI_Louvain,ARI_kmeans,ARI_HC,AMI_Louvain,AMI_kmeans,AMI_HC,Homogeneity_Louvain,Homogeneity_kmeans,Homogeneity_HC
ChromVAR_kmers,0.393487,0.260785,0.225031,0.524566,0.413008,0.356731,0.543092,0.370503,0.301594
ChromVAR_motifs,0.319824,0.209636,0.323051,0.510618,0.413654,0.464172,0.534893,0.429699,0.437689
cisTopic,0.551506,0.351849,0.383048,0.661205,0.555908,0.573953,0.674538,0.585062,0.59729
Cusanovich2018,0.490138,-0.00194164,-0.00194164,0.636935,-0.00278841,-0.00278841,0.635598,0.00302188,0.00302188
Control,0.183302,0.0229697,0.0397529,0.363688,0.0314137,0.0673721,0.377179,0.0328729,0.0616285
GeneScoring,0.0403609,0.0179964,0.0255461,0.111789,0.0318159,0.0369079,0.11883,0.0346644,0.0399724
Scasat,0.305007,0.161367,0.161388,0.517868,0.302171,0.378108,0.531279,0.320602,0.383529
scABC,0.020894,0.0109238,0.0327437,0.0737247,0.0190839,0.0682762,0.0780852,0.0196556,0.0635191
Cicero,0.033752,-0.00369183,-0.00194164,0.091265,0.0020334,-0.00278841,0.095604,0.00611831,0.00302188
ChromVAR_kmers_pca,0.431822,0.249719,0.238284,0.554208,0.413693,0.370762,0.57097,0.386348,0.313476
